<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/0704/04_model_saving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모델 저장하기

- 무엇을 저장해야 할까?
    - 모델의 구조
    - 모델의 weights

## 훈련하는 동안 체크포인트 저장하기

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
# 단순한 모델 정의
def create_model():
    model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
    ])

    model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

    return model

In [ ]:
model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


#### 체크포인트 콜백 사용하기

In [ ]:
# 체크포인트 파일 경로 설정
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path) # 현재 경로 폴더 반환
checkpoint_dir

'training_1'

In [ ]:
# 모델의 가중치를 저장하는 체크포인트 콜백 선언
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # 콜백 전달

Epoch 1/10
31/32 [============================>.] - ETA: 0s - loss: 1.1984 - accuracy: 0.6573
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 6s 118ms/step - loss: 1.1931 - accuracy: 0.6590 - val_loss: 0.7224 - val_accuracy: 0.7720
Epoch 2/10
30/32 [===========================>..] - ETA: 0s - loss: 0.4324 - accuracy: 0.8677
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 22ms/step - loss: 0.4266 - accuracy: 0.8720 - val_loss: 0.5325 - val_accuracy: 0.8270
Epoch 3/10
32/32 [==============================] - ETA: 0s - loss: 0.2828 - accuracy: 0.9300
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 13ms/step - loss: 0.2828 - accuracy: 0.9300 - val_loss: 0.4906 - val_accuracy: 0.8450
Epoch 4/10
31/32 [============================>.] - ETA: 0s - loss: 0.2074 - accuracy: 0.9516
Epoch 4: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 13ms/step - l

In [ ]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'checkpoint', 'cp.ckpt.index']

 텐서플로 체크포인트 파일을 만들고 에포크가 종료될 때마다 업데이트합니다  
 - 아키텍처가 동일한 모델 간에는 가중치를 공유 가능

 - 그렇다면, 원래 모델과 동일한 아키텍처를 가진 모델을 만든 다음 체크포인트 부여해보자.

In [ ]:
# 동일한 아키텍처를 가진 모델 선언
model = create_model()

# 체크포인트 주지말고, 한번 평가해보자.
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3156 - accuracy: 0.1350 - 275ms/epoch - 9ms/step
훈련되지 않은 모델의 정확도: 13.50%


허접하다.

In [ ]:
# 체크포인트에서 가중치를 로드하고 평가해보자.
model.load_weights(checkpoint_path)

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.3947 - accuracy: 0.8690 - 148ms/epoch - 5ms/step
복원된 모델의 정확도: 86.90%


In [ ]:
'{epoch:04d}, {batch:04d}'.format(batch=8, epoch=1)

'0001, 0008'

In [ ]:
# 파일 이름에 epoch 값 부여 (format 형식)
checkpoint_path = "training_2-1/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

- 체크포인트 주요 파라미터

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=True, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=True, # False면 모델 레이어 및 weight값 모두
    # save_freq=5 # 체크포인트 저장하는 주기 설정 가능 ('epoch'=에폭마다, 숫자=해당 배치마다)
    )

In [ ]:
# 새로운 모델 객체를 만듭니다
model = create_model()

In [ ]:
model.fit(train_images,
          train_labels,
          epochs=50,
          callbacks=[cp_callback], # 체크포인트 부여
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/50
32/32 [==============================] - 1s 16ms/step - loss: 1.1908 - accuracy: 0.6560 - val_loss: 0.7341 - val_accuracy: 0.7720
Epoch 2/50
32/32 [==============================] - 0s 11ms/step - loss: 0.4305 - accuracy: 0.8770 - val_loss: 0.5490 - val_accuracy: 0.8120
Epoch 3/50
32/32 [==============================] - 0s 14ms/step - loss: 0.2998 - accuracy: 0.9140 - val_loss: 0.4640 - val_accuracy: 0.8510
Epoch 4/50
32/32 [==============================] - 0s 9ms/step - loss: 0.2080 - accuracy: 0.9460 - val_loss: 0.4382 - val_accuracy: 0.8580
Epoch 5/50
32/32 [==============================] - 0s 12ms/step - loss: 0.1510 - accuracy: 0.9650 - val_loss: 0.4447 - val_accuracy: 0.8540
Epoch 6/50
32/32 [==============================] - 0s 10ms/step - loss: 0.1131 - accuracy: 0.9830 - val_loss: 0.4288 - val_accuracy: 0.8640
Epoch 7/50
32/32 [==============================] - 0s 13ms/step - loss: 0.0965 - accuracy: 0.9820 - val_loss: 0.4280 - val_accuracy: 0.8610
Epoch 8/50
32/

In [ ]:
# 동일한 아키텍처를 가진 모델 선언
model = create_model()

# 체크포인트 주지말고, 한번 평가해보자.
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3243 - accuracy: 0.1760 - 256ms/epoch - 8ms/step
훈련되지 않은 모델의 정확도: 17.60%


In [ ]:
# latest_checkpoint :최신값을 불러온다
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2-1/cp-0010.ckpt'

In [ ]:
model.load_weights(latest)

# 체크포인트 주지말고, 한번 평가해보자.
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.3948 - accuracy: 0.8740 - 82ms/epoch - 3ms/step
훈련되지 않은 모델의 정확도: 87.40%


In [ ]:
### mytraining1 폴더에 저장
### best only로 저장, 최종 값만 저장되도록
### weight값만
checkpoint_path = "mytraining1/cp.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=True, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=True, # False면 모델 레이어 및 weight값 모두
    # save_freq=5 # 체크포인트 저장하는 에폭주기 설정 가능
    )

model = create_model()

model.fit(train_images,
          train_labels,
          epochs=50,
          callbacks=[cp_callback], # 체크포인트 부여
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/50
32/32 [==============================] - 1s 16ms/step - loss: 1.1397 - accuracy: 0.6680 - val_loss: 0.7133 - val_accuracy: 0.7730
Epoch 2/50
32/32 [==============================] - 0s 12ms/step - loss: 0.4137 - accuracy: 0.8900 - val_loss: 0.5246 - val_accuracy: 0.8290
Epoch 3/50
32/32 [==============================] - 0s 12ms/step - loss: 0.2794 - accuracy: 0.9170 - val_loss: 0.4714 - val_accuracy: 0.8540
Epoch 4/50
32/32 [==============================] - 0s 10ms/step - loss: 0.2054 - accuracy: 0.9520 - val_loss: 0.4414 - val_accuracy: 0.8500
Epoch 5/50
32/32 [==============================] - 0s 10ms/step - loss: 0.1473 - accuracy: 0.9650 - val_loss: 0.4237 - val_accuracy: 0.8560
Epoch 6/50
32/32 [==============================] - 0s 9ms/step - loss: 0.1162 - accuracy: 0.9800 - val_loss: 0.4456 - val_accuracy: 0.8500
Epoch 7/50
32/32 [==============================] - 0s 12ms/step - loss: 0.0902 - accuracy: 0.9820 - val_loss: 0.4069 - val_accuracy: 0.8640
Epoch 8/50
32/

In [ ]:
model.load_weights(checkpoint_path)

model.evaluate(test_images,  test_labels, verbose=2)

32/32 - 0s - loss: 0.4008 - accuracy: 0.8690 - 93ms/epoch - 3ms/step


[0.40077367424964905, 0.8690000176429749]

In [ ]:
tf.keras.callbacks.ModelCheckpoint?

In [ ]:
### mytraining2 폴더에 저장
### 5epoch마다 저장
### weight값만
checkpoint_path = "mytraining3/cp-{epoch:04d}.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=False, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=True, # False면 모델 레이어 및 weight값 모두
    period=5, # 체크포인트 저장하는 에폭주기 설정 가능
    save_freq='epoch' # epoch의 끝에서 저장할지 여부
    )

In [ ]:
model = create_model()

model.fit(train_images,
          train_labels,
          epochs=50,
          callbacks=[cp_callback], # 체크포인트 부여
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/50


32/32 [==============================] - 1s 14ms/step - loss: 1.1627 - accuracy: 0.6790 - val_loss: 0.6930 - val_accuracy: 0.7870
Epoch 2/50
32/32 [==============================] - 0s 12ms/step - loss: 0.4093 - accuracy: 0.8850 - val_loss: 0.5362 - val_accuracy: 0.8210
Epoch 3/50
32/32 [==============================] - 0s 11ms/step - loss: 0.2728 - accuracy: 0.9390 - val_loss: 0.4898 - val_accuracy: 0.8430
Epoch 4/50
32/32 [==============================] - 0s 12ms/step - loss: 0.2152 - accuracy: 0.9490 - val_loss: 0.4306 - val_accuracy: 0.8510
Epoch 5/50
32/32 [==============================] - 0s 10ms/step - loss: 0.1464 - accuracy: 0.9710 - val_loss: 0.4222 - val_accuracy: 0.8590
Epoch 6/50
32/32 [==============================] - 0s 9ms/step - loss: 0.1217 - accuracy: 0.9700 - val_loss: 0.4322 - val_accuracy: 0.8660
Epoch 7/50
32/32 [==============================] - 0s 12ms/step - loss: 0.0885 - accuracy: 0.9830 - val_loss: 0.4018 - val_accuracy: 0.8670
Epoch 8/50
32/32 [=======

### early stopping & 체크포인트 같이 사용하기

In [ ]:
checkpoint_path = "training_3/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=True, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=True, # False면 모델 레이어 및 weight값 모두
    # save_freq=5 # 체크포인트 저장하는 에폭주기 설정 가능
    )

In [ ]:
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
# 새로운 모델 객체를 만듭니다
model = create_model()

In [ ]:
model.fit(train_images,
          train_labels,
          epochs=100,
          callbacks=[cp_callback, early], # 체크포인트 부여, early stopping
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/100
32/32 [==============================] - 1s 16ms/step - loss: 1.1715 - accuracy: 0.6750 - val_loss: 0.7166 - val_accuracy: 0.7920
Epoch 2/100
32/32 [==============================] - 0s 12ms/step - loss: 0.4363 - accuracy: 0.8700 - val_loss: 0.5351 - val_accuracy: 0.8380
Epoch 3/100
32/32 [==============================] - 0s 12ms/step - loss: 0.2764 - accuracy: 0.9320 - val_loss: 0.4942 - val_accuracy: 0.8460
Epoch 4/100
32/32 [==============================] - 0s 9ms/step - loss: 0.2045 - accuracy: 0.9480 - val_loss: 0.4646 - val_accuracy: 0.8420
Epoch 5/100
32/32 [==============================] - 0s 9ms/step - loss: 0.1539 - accuracy: 0.9750 - val_loss: 0.4359 - val_accuracy: 0.8510
Epoch 6/100
32/32 [==============================] - 0s 11ms/step - loss: 0.1072 - accuracy: 0.9870 - val_loss: 0.4268 - val_accuracy: 0.8630
Epoch 7/100
32/32 [==============================] - 0s 12ms/step - loss: 0.0803 - accuracy: 0.9920 - val_loss: 0.4230 - val_accuracy: 0.8680
Epoch 8/

### 직접 가중치 체크포인트 저장

In [ ]:
# 학습이 끝난 후에 가중치 저장, checkpoint 못썼을 때 사용
model.save_weights('./checkpoints/my_checkpoint.ckpt')

model = create_model()

# 가중치 복원
model.load_weights('./checkpoints/my_checkpoint.ckpt')

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4118 - accuracy: 0.8750 - 258ms/epoch - 8ms/step
복원된 모델의 정확도: 87.50%


## 전체 모델 저장(모델레이어와 함께 저장)
1) save_weights_only=False 로 변경  
2) 직접 model.save() : SavedModel(pb)  
3) HDF5 파일로 저장하기

### save_weights_only=False 로 변경

In [ ]:
checkpoint_path = "training_4/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=True, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=False, # False면 모델 레이어 및 weight값 모두
    # save_freq=5 # 체크포인트 저장하는 에폭주기 설정 가능
    )

In [ ]:
# 새로운 모델 객체를 만듭니다
model = create_model()

In [ ]:
model.fit(train_images,
          train_labels,
          epochs=1,
          callbacks=[cp_callback], # 체크포인트 부여
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/100
31/32 [============================>.] - ETA: 0s - loss: 1.1893 - accuracy: 0.6421

32/32 [==============================] - 2s 39ms/step - loss: 1.1825 - accuracy: 0.6440 - val_loss: 0.7005 - val_accuracy: 0.8000
Epoch 2/100
32/32 [==============================] - ETA: 0s - loss: 0.4217 - accuracy: 0.8800

32/32 [==============================] - 1s 29ms/step - loss: 0.4217 - accuracy: 0.8800 - val_loss: 0.5121 - val_accuracy: 0.8470
Epoch 3/100
31/32 [============================>.] - ETA: 0s - loss: 0.2834 - accuracy: 0.9264

32/32 [==============================] - 1s 31ms/step - loss: 0.2833 - accuracy: 0.9270 - val_loss: 0.4990 - val_accuracy: 0.8480
Epoch 4/100
31/32 [============================>.] - ETA: 0s - loss: 0.2025 - accuracy: 0.9486

32/32 [==============================] - 1s 29ms/step - loss: 0.2026 - accuracy: 0.9480 - val_loss: 0.4672 - val_accuracy: 0.8440
Epoch 5/100
32/32 [==============================] - ETA: 0s - loss: 0.1500 - accuracy: 0.9710

32/32 [==============================] - 1s 48ms/step - loss: 0.1500 - accuracy: 0.9710 - val_loss: 0.4158 - val_accuracy: 0.8620
Epoch 6/100
32/32 [==============================] - 0s 13ms/step - loss: 0.1069 - accuracy: 0.9880 - val_loss: 0.4549 - val_accuracy: 0.8460
Epoch 7/100
32/32 [==============================] - 1s 21ms/step - loss: 0.0818 - accuracy: 0.9880 - val_loss: 0.4415 - val_accuracy: 0.8620
Epoch 8/100
29/32 [==========================>...] - ETA: 0s - loss: 0.0661 - accuracy: 0.9903

32/32 [==============================] - 3s 89ms/step - loss: 0.0633 - accuracy: 0.9910 - val_loss: 0.3999 - val_accuracy: 0.8770
Epoch 9/100
32/32 [==============================] - 0s 13ms/step - loss: 0.0479 - accuracy: 0.9980 - val_loss: 0.4189 - val_accuracy: 0.8680
Epoch 10/100
31/32 [============================>.] - ETA: 0s - loss: 0.0435 - accuracy: 0.9970

32/32 [==============================] - 1s 30ms/step - loss: 0.0434 - accuracy: 0.9970 - val_loss: 0.3942 - val_accuracy: 0.8750
Epoch 11/100
32/32 [==============================] - 0s 11ms/step - loss: 0.0339 - accuracy: 1.0000 - val_loss: 0.4137 - val_accuracy: 0.8710
Epoch 12/100
32/32 [==============================] - 0s 10ms/step - loss: 0.0245 - accuracy: 1.0000 - val_loss: 0.4152 - val_accuracy: 0.8710
Epoch 13/100
32/32 [==============================] - 0s 14ms/step - loss: 0.0194 - accuracy: 1.0000 - val_loss: 0.4153 - val_accuracy: 0.8700
Epoch 14/100
32/32 [==============================] - 0s 13ms/step - loss: 0.0189 - accuracy: 0.9990 - val_loss: 0.4076 - val_accuracy: 0.8740
Epoch 15/100
32/32 [==============================] - 0s 11ms/step - loss: 0.0154 - accuracy: 1.0000 - val_loss: 0.4065 - val_accuracy: 0.8790
Epoch 16/100
32/32 [==============================] - 0s 13ms/step - loss: 0.0123 - accuracy: 1.0000 - val_loss: 0.4265 - val_accuracy: 0.8690
Epoch 17/100

KeyboardInterrupt: ignored

### SavedModel(pb)
SavedModel 형식은 protobuf 바이너리와 TensorFlow 체크포인트를 포함

In [ ]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# SavedModel로 전체 모델을 저장합니다
!mkdir -p saved_model # 폴더를 만들어주는 명령어
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 8ms/step - loss: 1.1501 - accuracy: 0.6600
Epoch 2/5
32/32 [==============================] - 0s 8ms/step - loss: 0.4241 - accuracy: 0.8760
Epoch 3/5
32/32 [==============================] - 0s 8ms/step - loss: 0.2846 - accuracy: 0.9200
Epoch 4/5
32/32 [==============================] - 0s 11ms/step - loss: 0.2242 - accuracy: 0.9480
Epoch 5/5
32/32 [==============================] - 0s 8ms/step - loss: 0.1560 - accuracy: 0.9670


In [ ]:
# my_model 디렉토리
!ls saved_model

my_model


In [ ]:
# assests 폴더, saved_model.pb, variables 폴더
!ls saved_model/my_model

assets	keras_metadata.pb  saved_model.pb  variables


In [ ]:
# pd파일 폴더에서 바로 모델을 불러올 수 있습니다(레이어까지 저장되어 있기 때문에)
new_model = tf.keras.models.load_model('saved_model/my_model')

# 모델 구조를 확인합니다
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


복원된 모델은 원본 모델과 동일한 매개변수로 컴파일되어 있습니다. 이 모델을 평가하고 예측에 사용해 보죠:

In [ ]:
# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4360 - accuracy: 0.8580 - 230ms/epoch - 7ms/step
복원된 모델의 정확도: 85.80%
32/32 [==============================] - 0s 3ms/step
(1000, 10)


### HDF5 파일로 저장하기
케라스는 HDF5 표준을 따르는 기본 저장 포맷을 제공합니다.

In [ ]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
# '.h5' 확장자는 이 모델이 HDF5로 저장되었다는 것을 나타냅니다
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 1s 7ms/step - loss: 1.1547 - accuracy: 0.6590
Epoch 2/5
32/32 [==============================] - 0s 6ms/step - loss: 0.4200 - accuracy: 0.8900
Epoch 3/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2866 - accuracy: 0.9280
Epoch 4/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2021 - accuracy: 0.9490
Epoch 5/5
32/32 [==============================] - 0s 6ms/step - loss: 0.1626 - accuracy: 0.9630


### h5 VS pb
- 공통점 : 레이어 & 가중치 모두 저장
- 차이점  
1) HDF5 : 객체 구성을 사용하여 모델 아키텍처를 저장  
2) SavedModel : 실행 그래프를 저장. 원본 코드 없이도 사용자 지정 객체를 저장 가능

#### h5 to pb 변환하기

In [ ]:
!mkdir converted

In [ ]:
model = tf.keras.models.load_model('my_model.h5')

In [ ]:
export_path = 'converted'
# save_format="tf" 는 tensorflow 형식으로 변경하겠다 = pb파일로 변경하겠다
model.save(export_path, save_format="tf")